In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn
import cv2
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

import tensorflow as tf
import keras
from keras.utils.np_utils import to_categorical
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout, Convolution2D,MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
from tqdm import tqdm
from PIL import Image

In [10]:
from keras.applications import VGG16

#Load the VGG model
image_size = 150
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

In [11]:
# Freeze the layers except the last 11 layers

for layer in vgg_conv.layers[:-11]:

    layer.trainable = False

# Check the trainable status of the individual layers

for layer in vgg_conv.layers:
    print(layer, layer.trainable)


<keras.engine.input_layer.InputLayer object at 0x000001D2E3F23888> False
<keras.layers.convolutional.Conv2D object at 0x000001D2E3CFD408> False
<keras.layers.convolutional.Conv2D object at 0x000001D2E3CFD248> False
<keras.layers.pooling.MaxPooling2D object at 0x000001D2E3F1DE08> False
<keras.layers.convolutional.Conv2D object at 0x000001D2E3CFBA48> False
<keras.layers.convolutional.Conv2D object at 0x000001D2E3F5CA48> False
<keras.layers.pooling.MaxPooling2D object at 0x000001D2E3F6ACC8> False
<keras.layers.convolutional.Conv2D object at 0x000001D2E3F72FC8> False
<keras.layers.convolutional.Conv2D object at 0x000001D2E3F7B9C8> True
<keras.layers.convolutional.Conv2D object at 0x000001D2E3F81D08> True
<keras.layers.pooling.MaxPooling2D object at 0x000001D2E3F8B348> True
<keras.layers.convolutional.Conv2D object at 0x000001D2E3F95348> True
<keras.layers.convolutional.Conv2D object at 0x000001D2E3F9AD08> True
<keras.layers.convolutional.Conv2D object at 0x000001D2E3FA2EC8> True
<keras.lay

In [15]:
from keras import models
from keras import layers
from keras import optimizers

# Create the model
model = models.Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))
# Show a summary of the model. Check the number of trainable parameters
model.summary()



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 2052      
Total params: 14,716,740
Trainable params: 14,161,412
Non-trainable params: 555,328
_________________________________________________________________


In [24]:
augmentor = ImageDataGenerator(rescale=1./255,zoom_range=0.2,shear_range=0.2,horizontal_flip=True)
train_generator = augmentor.flow_from_directory('D:/rdataset/use/Training',
                                            class_mode="categorical",
                                            target_size=(150, 150),
                                            color_mode="rgb",
                                            shuffle=True,
                                            batch_size = 32)

# initialize the validation generator
val_generator = augmentor.flow_from_directory('D:/rdataset/use/Validation',
                                         class_mode="categorical",
                                         target_size=(150, 150),
                                         color_mode="rgb",
                                         shuffle=False,
                                         batch_size = 32)

# initialize the testing generator
test_generator = augmentor.flow_from_directory('D:/rdataset/use/Testing',
                                     class_mode="categorical",
                                     target_size=(150, 150),
                                     color_mode="rgb",
                                     shuffle=False,
                                     batch_size = 32)

Found 2012 images belonging to 4 classes.
Found 672 images belonging to 4 classes.
Found 672 images belonging to 4 classes.


In [25]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])


In [26]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=70,
                              epochs=5,
                              validation_data=test_generator,
                              validation_steps=64)


Epoch 1/5
70/70 [==============================] - ETA: 41:22 - loss: 1.3817 - acc: 0.18 - ETA: 30:41 - loss: 1.3748 - acc: 0.25 - ETA: 29:24 - loss: 1.4044 - acc: 0.26 - ETA: 27:15 - loss: 1.4064 - acc: 0.22 - ETA: 26:05 - loss: 1.4012 - acc: 0.24 - ETA: 24:07 - loss: 1.4071 - acc: 0.26 - ETA: 23:58 - loss: 1.4043 - acc: 0.26 - ETA: 23:46 - loss: 1.4013 - acc: 0.26 - ETA: 23:43 - loss: 1.3985 - acc: 0.28 - ETA: 23:01 - loss: 1.3918 - acc: 0.28 - ETA: 22:59 - loss: 1.4107 - acc: 0.28 - ETA: 24:26 - loss: 1.4065 - acc: 0.28 - ETA: 24:11 - loss: 1.4062 - acc: 0.29 - ETA: 24:24 - loss: 1.4051 - acc: 0.29 - ETA: 23:55 - loss: 1.4044 - acc: 0.29 - ETA: 23:43 - loss: 1.4032 - acc: 0.29 - ETA: 23:13 - loss: 1.4025 - acc: 0.29 - ETA: 23:50 - loss: 1.4016 - acc: 0.28 - ETA: 24:10 - loss: 1.4006 - acc: 0.27 - ETA: 24:05 - loss: 1.3992 - acc: 0.28 - ETA: 23:36 - loss: 1.3939 - acc: 0.29 - ETA: 22:57 - loss: 1.4110 - acc: 0.28 - ETA: 22:36 - loss: 1.4150 - acc: 0.28 - ETA: 22:04 - loss: 1.4173 - a

In [29]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=10,
                              epochs=20,
                              validation_data=val_generator,
                              validation_steps=10,
                              callbacks=[checkpoint]
                             )

Epoch 1/20
10/10 [==============================] - ETA: 8:17 - loss: 1.4913 - acc: 0.312 - ETA: 7:11 - loss: 1.3329 - acc: 0.468 - ETA: 5:41 - loss: 1.2672 - acc: 0.531 - ETA: 4:47 - loss: 1.2828 - acc: 0.500 - ETA: 3:52 - loss: 1.2431 - acc: 0.537 - ETA: 3:03 - loss: 1.2310 - acc: 0.531 - ETA: 2:21 - loss: 1.2031 - acc: 0.544 - ETA: 1:33 - loss: 1.1833 - acc: 0.550 - ETA: 46s - loss: 1.1615 - acc: 0.555 - 754s 75s/step - loss: 1.1527 - acc: 0.5531 - val_loss: 1.0952 - val_acc: 0.5813

Epoch 00001: val_loss improved from inf to 1.09518, saving model to new_model.h5
Epoch 2/20
10/10 [==============================] - ETA: 5:00 - loss: 1.6108 - acc: 0.500 - ETA: 3:44 - loss: 1.4780 - acc: 0.484 - ETA: 3:32 - loss: 1.4070 - acc: 0.510 - ETA: 3:15 - loss: 1.3646 - acc: 0.500 - ETA: 2:53 - loss: 1.3212 - acc: 0.500 - ETA: 2:31 - loss: 1.2827 - acc: 0.536 - ETA: 1:45 - loss: 1.2671 - acc: 0.536 - ETA: 1:09 - loss: 1.2557 - acc: 0.531 - ETA: 34s - loss: 1.2554 - acc: 0.531 - 588s 59s/step - 

In [27]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('new_model.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='auto') 